In [2]:
%load_ext autoreload
%autoreload 2
from indices import MainDataset
from neighbors import Neighborhoods
import functions
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from catboost import Pool
from metric import metrics_stat

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
train_dataset = MainDataset("dataset/train.csv")
test_dataset = MainDataset("dataset/test.csv", need_index = False)

/Users/ivan.bragin/projects/raif/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [4]:
neighborhoods = Neighborhoods(train_dataset.index)
neighborhoods1 = Neighborhoods(train_dataset.index1)

In [5]:
# for i, o in enumerate(train_dataset.all_objects[:10000]):
#     if i % 1000 == 0:
#         print(i)
#     neighbor = neighborhoods.get_haversine_closest(o, 7)
#     for f_name, f in functions.items():
#         prices = [f(o, x[0], x[1]) for x in neighbor]
#         o.neighbors_scores[f_name] = np.mean(prices)

In [6]:
for prefix, neighborhoods_item in [("", neighborhoods), ("P1_", neighborhoods1)]:
    for i, o in enumerate(train_dataset.all_objects):
#         o.neighbors_scores = {}
        if o.row["price_type"] != 1:
            continue
        if i % 1000 == 0:
            print(i)
        neighbor = neighborhoods_item.get_haversine_closest(o, 7)
        for f_name, f in functions.functions.items():
            prices = [f(o, x[0], x[1]) for x in neighbor]
            prices = [x for x in prices if x > 0]
            if len(prices) > 0:
                price_median = np.median(prices)
                price_max = np.max(prices)
                price_min = np.min(prices)

            else:
                price_median = 0
                price_max = 0
                price_min = 0
            o.neighbors_scores[prefix + f_name+"_median"] = price_median
            o.neighbors_scores[prefix + f_name+"_max"] = price_max
            o.neighbors_scores[prefix + f_name+"_min"] = price_min
    

253000
254000
255000
256000
253000
254000
255000
256000


In [7]:
for prefix, neighborhoods_item in [("", neighborhoods), ("P1_", neighborhoods1)]:
    for i, o in enumerate(test_dataset.all_objects):
        if o.row["price_type"] != 1:
            continue
        if i % 1000 == 0:
            print(i)
        neighbor = neighborhoods_item.get_haversine_closest(o, 7)
        for f_name, f in functions.functions.items():
            prices = [f(o, x[0], x[1]) for x in neighbor]
            prices = [x for x in prices if x > 0]
            if len(prices) > 0:
                price_median = np.median(prices)
                price_max = np.max(prices)
                price_min = np.min(prices)

            else:
                price_median = 0
                price_max = 0
                price_min = 0
            o.neighbors_scores[prefix + f_name+"_median"] = price_median
            o.neighbors_scores[prefix + f_name+"_max"] = price_max
            o.neighbors_scores[prefix + f_name+"_min"] = price_min

0
1000
2000
0
1000
2000


In [8]:
raw_dense_features = [
#     'osm_historic_points_in_0.005',
#     'osm_culture_points_in_0.0075',
        'osm_amenity_points_in_0.001',
       'osm_amenity_points_in_0.005', 'osm_amenity_points_in_0.0075',
       'osm_amenity_points_in_0.01', 'osm_building_points_in_0.001',
       'osm_building_points_in_0.005', 'osm_building_points_in_0.0075',
       'osm_building_points_in_0.01', 'osm_catering_points_in_0.001',
       'osm_catering_points_in_0.005', 'osm_catering_points_in_0.0075',
       'osm_catering_points_in_0.01', 'osm_city_closest_dist',
        'osm_city_nearest_population',
       'osm_crossing_closest_dist', 'osm_crossing_points_in_0.001',
       'osm_crossing_points_in_0.005', 'osm_crossing_points_in_0.0075',
       'osm_crossing_points_in_0.01', 'osm_culture_points_in_0.001',
       'osm_culture_points_in_0.005', 
       'osm_culture_points_in_0.01', 'osm_finance_points_in_0.001',
       'osm_finance_points_in_0.005', 'osm_finance_points_in_0.0075',
       'osm_finance_points_in_0.01', 'osm_healthcare_points_in_0.005',
       'osm_healthcare_points_in_0.0075', 'osm_healthcare_points_in_0.01',
        'osm_historic_points_in_0.0075',
       'osm_historic_points_in_0.01', 'osm_hotels_points_in_0.005',
       'osm_hotels_points_in_0.0075', 'osm_hotels_points_in_0.01',
       'osm_leisure_points_in_0.005', 'osm_leisure_points_in_0.0075',
       'osm_leisure_points_in_0.01', 'osm_offices_points_in_0.001',
       'osm_offices_points_in_0.005', 'osm_offices_points_in_0.0075',
       'osm_offices_points_in_0.01', 'osm_shops_points_in_0.001',
       'osm_shops_points_in_0.005', 'osm_shops_points_in_0.0075',
       'osm_shops_points_in_0.01', 'osm_subway_closest_dist',
       'osm_train_stop_closest_dist', 'osm_train_stop_points_in_0.005',
       'osm_train_stop_points_in_0.0075', 'osm_train_stop_points_in_0.01',
       'osm_transport_stop_closest_dist', 'osm_transport_stop_points_in_0.005',
       'osm_transport_stop_points_in_0.0075',
       'osm_transport_stop_points_in_0.01',
                      'reform_count_of_houses_1000', 'reform_count_of_houses_500',
       'reform_house_population_1000', 'reform_house_population_500',
       'reform_mean_floor_count_1000', 'reform_mean_floor_count_500',
       'reform_mean_year_building_1000', 'reform_mean_year_building_500',
       'total_square', "lat", "lng"
       ]

raw_cat_features = ['city', 'osm_city_nearest_name', 'region', 'realty_type', 'street', 'floor']

bad_f = set(["F_square_False_min", "F_square_False_median"])
def getX(o):
    x = []
    for f_name, score in o.neighbors_scores.items():
        if f_name not in bad_f and "P1_" not in f_name:
            x.append(score)
    for n in raw_dense_features:
        x.append(o.row[n])
    for n in raw_cat_features:
        x.append(o.row[n])
    return pd.Series(x)


In [9]:
columns = []
for p in [""
#           "P1_"
         ]:
    for t in ["_median", "_min", "_max"]:
        columns.extend([p + f_name + t for f_name, f in functions.functions.items() ])
columns = [c for c in columns if c not in bad_f]
    # columns = []
columns.extend(raw_dense_features)
columns.extend(raw_cat_features)

In [10]:
# X0 = [getX(o) for o in train_dataset.all_objects[:1] if o.row["price_type"] == 0]
# X0 = pd.concat(X0, axis=1).T
# X0.columns=columns
# Y0 = [o.getY() for o in train_dataset.all_objects if o.row["price_type"] == 0]
# X0["floor"] = X0["floor"].astype("str")

In [11]:
X1 = [getX(o) for o in train_dataset.all_objects if o.row["price_type"] == 1]
X1 = pd.concat(X1, axis=1).T
X1.columns=columns
Y1 = [o.getY() for o in train_dataset.all_objects if o.row["price_type"] == 1]
X1 = X1.fillna(-999)


In [12]:
Xtest1 = [getX(o) for o in test_dataset.all_objects]
Xtest1 = pd.concat(Xtest1, axis=1).T
Xtest1.columns=columns
Xtest1 = Xtest1.fillna(-999)

In [13]:
# X0 = np.stack([o.getX() for o in train_dataset.all_objects if o.row["price_type"] == 0])
# X1 = np.stack([o.getX() for o in train_dataset.all_objects if o.row["price_type"] == 1])
# Y1 = [o.getY() for o in train_dataset.all_objects if o.row["price_type"] == 1]

In [14]:
# np.save("x0", X0)
# np.save("y0", Y0)
# np.save("x1", X1)
# np.save("y1", Y1)

In [33]:
# X0 = X0.fillna(-999)
X_train, X_val, Y_train, Y_val = train_test_split(X1, Y1, test_size=0.1, random_state=42)

In [34]:

train_data = Pool(data=X1,
                  cat_features=raw_cat_features,
                  label=Y1)

val_data = Pool(data=X_val,
                cat_features=raw_cat_features,
                  label=Y_val)

In [35]:
cat_model = CatBoostRegressor(
    learning_rate=0.03,
    iterations=7000
)
cat_model.fit(train_data,eval_set=val_data, use_best_model=False)



Custom logger is already specified. Specify more than one logger at same time is not thread safe.

0:	learn: 88470.2297825	test: 83238.8995672	best: 83238.8995672 (0)	total: 11.2ms	remaining: 1m 18s
1:	learn: 87670.7343123	test: 82088.7878396	best: 82088.7878396 (1)	total: 22ms	remaining: 1m 17s
2:	learn: 86927.5718519	test: 81119.3592836	best: 81119.3592836 (2)	total: 33.1ms	remaining: 1m 17s
3:	learn: 86170.9590886	test: 79889.2602760	best: 79889.2602760 (3)	total: 43ms	remaining: 1m 15s
4:	learn: 85356.1003857	test: 78816.2407543	best: 78816.2407543 (4)	total: 53.3ms	remaining: 1m 14s
5:	learn: 84598.0887360	test: 77712.4874392	best: 77712.4874392 (5)	total: 63.2ms	remaining: 1m 13s
6:	learn: 83862.3472375	test: 76955.2541564	best: 76955.2541564 (6)	total: 73.3ms	remaining: 1m 13s
7:	learn: 83199.4317747	test: 75911.3663290	best: 75911.3663290 (7)	total: 83.6ms	remaining: 1m 13s
8:	learn: 82492.8863351	test: 74739.1291691	best: 74739.1291691 (8)	total: 95ms	remaining: 1m 13s
9:	learn: 81780.8892661	test: 73603.9892702	best: 73603.9892702 (9)	total: 106ms	remaining: 1m 13s
10:	lea

82:	learn: 61128.8298108	test: 41111.8646888	best: 41111.8646888 (82)	total: 1.02s	remaining: 1m 25s
83:	learn: 61057.6917572	test: 40909.6983953	best: 40909.6983953 (83)	total: 1.03s	remaining: 1m 24s
84:	learn: 60908.5736129	test: 40661.3760181	best: 40661.3760181 (84)	total: 1.04s	remaining: 1m 24s
85:	learn: 60864.9765482	test: 40538.9787877	best: 40538.9787877 (85)	total: 1.05s	remaining: 1m 24s
86:	learn: 60460.3073355	test: 40340.1337285	best: 40340.1337285 (86)	total: 1.07s	remaining: 1m 24s
87:	learn: 60406.5550527	test: 40265.1248908	best: 40265.1248908 (87)	total: 1.08s	remaining: 1m 24s
88:	learn: 60309.7860596	test: 40197.3750024	best: 40197.3750024 (88)	total: 1.09s	remaining: 1m 24s
89:	learn: 60190.8085605	test: 40009.9474314	best: 40009.9474314 (89)	total: 1.1s	remaining: 1m 24s
90:	learn: 60104.8712271	test: 39967.0872108	best: 39967.0872108 (90)	total: 1.11s	remaining: 1m 24s
91:	learn: 60010.8529089	test: 39799.0096039	best: 39799.0096039 (91)	total: 1.13s	remaining

176:	learn: 52222.6491932	test: 34130.7551595	best: 34130.7551595 (176)	total: 2.01s	remaining: 1m 17s
177:	learn: 52189.2265585	test: 34061.5328517	best: 34061.5328517 (177)	total: 2.02s	remaining: 1m 17s
178:	learn: 51998.1972802	test: 34033.3480539	best: 34033.3480539 (178)	total: 2.03s	remaining: 1m 17s
179:	learn: 51944.5908964	test: 34009.0399612	best: 34009.0399612 (179)	total: 2.04s	remaining: 1m 17s
180:	learn: 51908.0486106	test: 33981.7601500	best: 33981.7601500 (180)	total: 2.05s	remaining: 1m 17s
181:	learn: 51849.8627782	test: 33900.0703547	best: 33900.0703547 (181)	total: 2.06s	remaining: 1m 17s
182:	learn: 51726.5353650	test: 33890.7998656	best: 33890.7998656 (182)	total: 2.07s	remaining: 1m 16s
183:	learn: 51637.6570050	test: 33842.5937493	best: 33842.5937493 (183)	total: 2.08s	remaining: 1m 16s
184:	learn: 51555.4807841	test: 33826.2706040	best: 33826.2706040 (184)	total: 2.08s	remaining: 1m 16s
185:	learn: 51528.2910702	test: 33790.7470632	best: 33790.7470632 (185)	t

256:	learn: 47291.0794343	test: 31327.2657696	best: 31324.9103693 (254)	total: 2.81s	remaining: 1m 13s
257:	learn: 47252.4461612	test: 31299.2949237	best: 31299.2949237 (257)	total: 2.82s	remaining: 1m 13s
258:	learn: 47175.1668162	test: 31285.5936232	best: 31285.5936232 (258)	total: 2.83s	remaining: 1m 13s
259:	learn: 46962.4338748	test: 31284.2986300	best: 31284.2986300 (259)	total: 2.84s	remaining: 1m 13s
260:	learn: 46932.2042248	test: 31276.9477557	best: 31276.9477557 (260)	total: 2.85s	remaining: 1m 13s
261:	learn: 46918.1510196	test: 31208.6828005	best: 31208.6828005 (261)	total: 2.86s	remaining: 1m 13s
262:	learn: 46621.8521013	test: 31194.0597383	best: 31194.0597383 (262)	total: 2.87s	remaining: 1m 13s
263:	learn: 46544.3122492	test: 31105.5435270	best: 31105.5435270 (263)	total: 2.88s	remaining: 1m 13s
264:	learn: 46517.2893731	test: 31063.8391252	best: 31063.8391252 (264)	total: 2.89s	remaining: 1m 13s
265:	learn: 46204.5698421	test: 31053.8660823	best: 31053.8660823 (265)	t

...

6637:	learn: 4699.1975541	test: 6104.6394793	best: 6103.8473887 (6634)	total: 1m 17s	remaining: 4.2s
6638:	learn: 4697.9149193	test: 6103.6116038	best: 6103.6116038 (6638)	total: 1m 17s	remaining: 4.19s
6639:	learn: 4697.1581497	test: 6102.9760192	best: 6102.9760192 (6639)	total: 1m 17s	remaining: 4.18s
6640:	learn: 4696.2453431	test: 6102.1881429	best: 6102.1881429 (6640)	total: 1m 17s	remaining: 4.17s
6641:	learn: 4696.1695215	test: 6102.6433703	best: 6102.1881429 (6640)	total: 1m 17s	remaining: 4.15s
6642:	learn: 4695.5398976	test: 6102.4418699	best: 6102.1881429 (6640)	total: 1m 17s	remaining: 4.14s
6643:	learn: 4694.4541486	test: 6101.9412281	best: 6101.9412281 (6643)	total: 1m 17s	remaining: 4.13s
6644:	learn: 4693.6896626	test: 6101.9161076	best: 6101.9161076 (6644)	total: 1m 17s	remaining: 4.12s
6645:	learn: 4692.6607887	test: 6101.1062572	best: 6101.1062572 (6645)	total: 1m 17s	remaining: 4.11s
6646:	learn: 4691.6836770	test: 6100.3948914	best: 6100.3948914 (6646)	total: 1m 17

6719:	learn: 4629.7262444	test: 6072.9521034	best: 6072.9521034 (6719)	total: 1m 18s	remaining: 3.27s
6720:	learn: 4629.1005599	test: 6073.5216148	best: 6072.9521034 (6719)	total: 1m 18s	remaining: 3.26s
6721:	learn: 4627.7179100	test: 6072.8221130	best: 6072.8221130 (6721)	total: 1m 18s	remaining: 3.25s
6722:	learn: 4627.3532856	test: 6072.4480544	best: 6072.4480544 (6722)	total: 1m 18s	remaining: 3.23s
6723:	learn: 4626.5251697	test: 6072.6531618	best: 6072.4480544 (6722)	total: 1m 18s	remaining: 3.22s
6724:	learn: 4625.7939252	test: 6072.4374310	best: 6072.4374310 (6724)	total: 1m 18s	remaining: 3.21s
6725:	learn: 4625.2064126	test: 6071.9758706	best: 6071.9758706 (6725)	total: 1m 18s	remaining: 3.2s
6726:	learn: 4624.9922285	test: 6072.1613462	best: 6071.9758706 (6725)	total: 1m 18s	remaining: 3.19s
6727:	learn: 4623.2295812	test: 6071.4868460	best: 6071.4868460 (6727)	total: 1m 18s	remaining: 3.18s
6728:	learn: 4621.5165061	test: 6069.1917167	best: 6069.1917167 (6728)	total: 1m 18

6802:	learn: 4560.1901073	test: 6030.4084834	best: 6030.4084834 (6802)	total: 1m 19s	remaining: 2.31s
6803:	learn: 4559.5222817	test: 6029.6892602	best: 6029.6892602 (6803)	total: 1m 19s	remaining: 2.3s
6804:	learn: 4558.7956303	test: 6029.3670818	best: 6029.3670818 (6804)	total: 1m 19s	remaining: 2.29s
6805:	learn: 4558.6148859	test: 6029.3511682	best: 6029.3511682 (6805)	total: 1m 19s	remaining: 2.28s
6806:	learn: 4558.5605020	test: 6029.8717541	best: 6029.3511682 (6805)	total: 1m 19s	remaining: 2.27s
6807:	learn: 4557.3327314	test: 6028.4713417	best: 6028.4713417 (6807)	total: 1m 19s	remaining: 2.25s
6808:	learn: 4556.0608076	test: 6028.6435970	best: 6028.4713417 (6807)	total: 1m 19s	remaining: 2.24s
6809:	learn: 4554.6756707	test: 6026.7464569	best: 6026.7464569 (6809)	total: 1m 19s	remaining: 2.23s
6810:	learn: 4554.4861541	test: 6026.3531595	best: 6026.3531595 (6810)	total: 1m 19s	remaining: 2.22s
6811:	learn: 4553.2118443	test: 6025.7663919	best: 6025.7663919 (6811)	total: 1m 19

6890:	learn: 4493.3876823	test: 5993.7593962	best: 5993.7593962 (6890)	total: 1m 21s	remaining: 1.28s
6891:	learn: 4491.8447887	test: 5993.5347860	best: 5993.5347860 (6891)	total: 1m 21s	remaining: 1.27s
6892:	learn: 4490.7614761	test: 5992.7749018	best: 5992.7749018 (6892)	total: 1m 21s	remaining: 1.26s
6893:	learn: 4489.4113152	test: 5992.1573150	best: 5992.1573150 (6893)	total: 1m 21s	remaining: 1.25s
6894:	learn: 4488.9279745	test: 5991.0924126	best: 5991.0924126 (6894)	total: 1m 21s	remaining: 1.23s
6895:	learn: 4488.1255457	test: 5990.5136742	best: 5990.5136742 (6895)	total: 1m 21s	remaining: 1.22s
6896:	learn: 4487.8592053	test: 5990.8242490	best: 5990.5136742 (6895)	total: 1m 21s	remaining: 1.21s
6897:	learn: 4487.5828664	test: 5990.7467983	best: 5990.5136742 (6895)	total: 1m 21s	remaining: 1.2s
6898:	learn: 4487.1571123	test: 5990.2054811	best: 5990.2054811 (6898)	total: 1m 21s	remaining: 1.19s
6899:	learn: 4487.0949766	test: 5990.1594465	best: 5990.1594465 (6899)	total: 1m 21

6985:	learn: 4423.1686348	test: 5950.7335048	best: 5950.7335048 (6985)	total: 1m 22s	remaining: 164ms
6986:	learn: 4422.4206827	test: 5949.3630423	best: 5949.3630423 (6986)	total: 1m 22s	remaining: 153ms
6987:	learn: 4421.4690347	test: 5949.1898601	best: 5949.1898601 (6987)	total: 1m 22s	remaining: 141ms
6988:	learn: 4420.1583526	test: 5947.7270928	best: 5947.7270928 (6988)	total: 1m 22s	remaining: 129ms
6989:	learn: 4419.0541770	test: 5947.0055967	best: 5947.0055967 (6989)	total: 1m 22s	remaining: 117ms
6990:	learn: 4418.6242169	test: 5946.3251192	best: 5946.3251192 (6990)	total: 1m 22s	remaining: 106ms
6991:	learn: 4417.9745844	test: 5945.7765689	best: 5945.7765689 (6991)	total: 1m 22s	remaining: 93.9ms
6992:	learn: 4416.5876032	test: 5943.0120255	best: 5943.0120255 (6992)	total: 1m 22s	remaining: 82.2ms
6993:	learn: 4415.4819466	test: 5941.9988885	best: 5941.9988885 (6993)	total: 1m 22s	remaining: 70.5ms
6994:	learn: 4414.2419948	test: 5941.4676428	best: 5941.4676428 (6994)	total: 1

In [36]:
pd.set_option("display.max_rows", None) 
cat_model.get_feature_importance(data=val_data,prettified=True)


,Feature Id,Importances
0,osm_crossing_points_in_0.005,6.169895
1,osm_hotels_points_in_0.005,4.696490
2,region,3.832049
3,total_square,3.300808
4,osm_catering_points_in_0.0075,3.137033
5,osm_building_points_in_0.005,2.950523
6,reform_mean_floor_count_500,2.858257
7,osm_amenity_points_in_0.005,2.841132
8,osm_catering_points_in_0.01,2.703656
9,F_max_distance_2_median,2.623662


In [37]:
price0_preds = cat_model.predict(X1)
metrics_stat(np.array(price0_preds) , np.array(Y1))

{'mape': 0.07673111483411932,
 'mdape': 0.05774004706207523,
 'rmse': 9830.03739367612,
 'r2': 0.9865369871866926,
 'raif_metric': 0.04776508377484689}

In [38]:
price0_preds = cat_model.predict(X_val, ntree_end=7000)
metrics_stat(np.array(Y_val), np.array(price0_preds))

{'mape': 0.08032333773167791,
 'mdape': 0.05903997897725699,
 'rmse': 5940.643756817104,
 'r2': 0.9950577564135158,
 'raif_metric': 0.06965755965538471}

In [ ]:
{'mape': 0.2556107178478481,
 'mdape': 0.1768883452314601,
 'rmse': 39604.04946249307,
 'r2': 0.7803474166348962,
 'raif_metric': 1.318478913210778}

In [ ]:
{'mape': 0.3938137450820849,
 'mdape': 0.27994944633239843,
 'rmse': 93738.73635776664,
 'r2': -3.7693751005637317,
 'raif_metric': 2.492385311141158}

{'mape': 0.28628533681632,
 'mdape': 0.21764848060697264,
 'rmse': 84390.73853846012,
 'r2': -1.6484418274393575,
 'raif_metric': 1.552534528099827}

In [ ]:
{'mape': 0.4006764913433361,
 'mdape': 0.28324961990050085,
 'rmse': 94343.39152646794,
 'r2': -4.286807955207906,
 'raif_metric': 2.6756861412533905}

{'mape': 0.2994072204832106,
 'mdape': 0.23136175216191418,
 'rmse': 85205.80509381538,
 'r2': -1.798582387807691,
 'raif_metric': 1.7313695017675064}

In [ ]:
{'mape': 0.45565440900238763,
 'mdape': 0.30249343647803606,
 'rmse': 100782.64525168539,
 'r2': -9.279126502012321,
 'raif_metric': 3.080882843995905}

In [72]:
X1.iloc[:,126:]

,realty_type,street,floor
0,110,S5845,-999
1,10,S27968,-999
2,10,S23597,-999
3,10,S10587,-999
4,10,S11220,-999
5,10,S6752,2.0
6,10,S6703,-999
7,10,S27593,-999
8,10,S19643,-999
9,10,S28820,-999


In [39]:
{'mape': 0.22629749264385002,
 'mdape': 0.19447444760686572,
 'rmse': 37925.78682309723,
 'r2': 0.6907862804815164,
 'raif_metric': 1.2450550091553048}
Xtest1 = Xtest1.fillna(-999)
test_data = Pool(data=Xtest1,
                  cat_features=raw_cat_features)
price0_preds = cat_model.predict(test_data)-2000

In [40]:
price0_preds

array([45462.14392876, 39790.12965627, 35693.80811542, ...,
       57991.99954738, 47212.54945654, 52237.91169646])

In [41]:
pd.DataFrame({"id": [o.row['id'] for o in test_dataset.all_objects],
             "per_square_meter_price": price0_preds
             }).to_csv("reduce.csv", index=False)